In [2]:
# 리밸런싱 
# out-of-sample에서 리밸런싱 우선
# 매달 리밸런싱 진행
    # 리밸런싱은 해당 기간 최적화 돌려서 나온 비중을 갖고 '매달' 진행.
    # 1. (out of sample 기간) 2023.05.01에 투자를 시작할 때 과거 5년치 데이터로 최적화 문제 풀고, 다음 한달간 수익률 보기. 
    # 그리고 2023.06.01에 시작할 때는 그때까지 데이터로 최적화 풀고, 
    # 나온 가중치로 리밸런싱 하고 한달간 수익률 보기… 
    # 반복
# 2. look back 기간을 1년, 2년, 등 여러 기간으로 설정하고 돌려보기.

In [3]:
# pip install pyPortfolioOpt
# pip install cplex
# pip install numpy
# pip install pandas
# pip install cvxpy

In [4]:
import numpy as np
import cvxpy as cp
import cplex
from tqdm import tqdm
from datetime import timedelta

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as pdr
import yfinance as yf

from dateutil.relativedelta import relativedelta
from datetime import datetime

In [5]:
# GPT 종목 (GPT weight)
stocks_15 = {
    "AAPL": 0.084, "AMZN": 0.074, "NVDA": 0.064, "JPM": 0.054, "PG": 0.054,
    "PFE": 0.064, "JNJ": 0.064, "KO": 0.054, "XOM": 0.064, "NEE": 0.074,
    "GOOGL": 0.084, "MSFT": 0.084, "TSLA": 0.074, "NKE": 0.054, "BAC": 0.054
}

stocks_30 = {
    "MSFT": 0.067, "AMZN": 0.067, "NVDA": 0.067, "AAPL": 0.067, "GOOGL": 0.067,
    "ADBE": 0.067, "JNJ": 0.05, "PFE": 0.05, "MRK": 0.05, "ABT": 0.05, 
    "PG": 0.05, "KO": 0.05, "JPM": 0.05, "GS": 0.05, "CAT": 0.025, 
    "CVX": 0.025, "XOM": 0.025, "BA": 0.025, "TSLA": 0.025, "NEE": 0.025, 
    "NKE": 0.005, "VZ": 0.005, "CRM": 0.005, "UNH": 0.005, "WMT": 0.005, 
    "QCOM": 0.005, "BAC": 0.005, "V": 0.005, "MCD": 0.005, "INTC": 0.005
}

stocks_45 = {
    "AAPL": 0.05, "GOOGL": 0.04, "MSFT": 0.04, "NVDA": 0.03, "AMD": 0.03, 
    "ORCL": 0.02, "CRM": 0.02, "INTC": 0.01, "CSCO": 0.01, "JPM": 0.04, 
    "GS": 0.03, "BAC": 0.03, "MS": 0.02, "AXP": 0.02, "C": 0.01,
    "JNJ": 0.03, "UNH": 0.03, "PFE": 0.02, "ABBV": 0.02, "TSLA": 0.02, 
    "AMGN": 0.02, "GILD": 0.01, "PG": 0.03, "KO": 0.03, "NKE": 0.02, 
    "PEP": 0.02, "COST": 0.02, "WMT": 0.02, "TGT": 0.01, "XOM": 0.025, 
    "CVX": 0.025, "NEE": 0.02, "DUK": 0.01, "SO": 0.01, "SLB": 0.01,
    "MMM": 0.02, "CAT": 0.02, "HON": 0.02, "GE": 0.02, "ADP": 0.02,
    "AMZN": 0.02, "META": 0.02, "HD": 0.02, "VZ": 0.01, "MRK": 0.01
}

In [6]:
# 종목명 리스트
tickers_15 = list(stocks_15.keys())
tickers_30 = list(stocks_30.keys())
tickers_45 = list(stocks_45.keys())

### GPT weight

In [7]:
gpt_15_weights = stocks_15
gpt_30_weights = stocks_30
gpt_45_weights = stocks_45

### Equal weight

In [8]:
eq_15_weights = [1/15] * 15
eq_30_weights = [1/30] * 30
eq_45_weights = [1/45] * 45

eq_15 =dict(zip(tickers_15, eq_15_weights))
eq_30 =dict(zip(tickers_30, eq_30_weights))
eq_45 =dict(zip(tickers_45, eq_45_weights))

### GMV + EPO weight

In [9]:
gmv_epo_15_weights = [0.049605485006088634, 0.07849495425750663, 0.06333929068001071, 0.05700662407849795, 0.07650289062642221, 0.05900628444525451, 0.057323863755786, 0.0475050526473884, 0.06851785110258568, 0.0551095984517666, 0.0635746953436858, 0.08684669491598282, 0.06555603211482636, 0.08160761317658853, 0.090003069397609]
gmv_epo_30_weights = [0.016666779461997225, 0.028582382826203727, 0.03181481366190954, 0.04612454337672658, 0.03430208600932986, 0.02496457702197057, 0.034559689579773685, 0.03603933972602465, 0.0348256115466567, 0.02719368909866466, 0.01666666768426058, 0.03550281897656019, 0.03316982644195968, 0.02106295226563651, 0.019372147251594442, 0.01873074735009172, 0.06623366706426345, 0.016666690262465863, 0.03423317149469612, 0.02304327143120648, 0.031214508772612753, 0.045807643648171685, 0.027946448832753296, 0.049668167222889026, 0.049588148769328606, 0.024769224302769302, 0.016666667674594095, 0.05283625719167181, 0.05583979037764232, 0.045907670675591185]
gmv_epo_45_weights = [0.011111111975729409, 0.026646602204962858, 0.011111111299922065, 0.0309210193238956, 0.025007258123960922, 0.034659892275724705, 0.01111111146901072, 0.011111111747740477, 0.011111111516746397, 0.01779091912009698, 0.01984888218960698, 0.02522647335013622, 0.011180137115527101, 0.017489649124790285, 0.028191045792364738, 0.03631158995254371, 0.044444444090877805, 0.016302194182152667, 0.011111111266490585, 0.011111111595258412, 0.01111111125737019, 0.022827040798679366, 0.020587601244674014, 0.011111111503325005, 0.011111111556128327, 0.044444443639370106, 0.011111111461049875, 0.04444444402780661, 0.011111111368812045, 0.011111112285545484, 0.023035029376855205, 0.01260693000477375, 0.02165124524507929, 0.02165119066334294, 0.011286898608143595, 0.029528803525066108, 0.017506358987491956, 0.03972350417693582, 0.018956379525549516, 0.036275899230639834, 0.040688577981395774, 0.011896549529971394, 0.03665347882393091, 0.03968317677255412, 0.028087890687943526]

gmv_epo_15 = dict(zip(tickers_15, gmv_epo_15_weights))
gmv_epo_30 = dict(zip(tickers_30, gmv_epo_30_weights))
gmv_epo_45 = dict(zip(tickers_45, gmv_epo_45_weights))

### MAX Sharpe + EPO weight

In [10]:
msp_epo_15_weights = [0.13333, 0.03333, 0.13333, 0.03333, 0.09539, 0.03333, 0.03333, 0.03333, 0.03707, 0.05699, 0.03897, 0.13333, 0.13333, 0.03825, 0.03333]
msp_epo_30_weights = [0.06667, 0.01667, 0.06667, 0.06667, 0.0387, 0.01667, 0.01667, 0.01667, 0.06667, 0.02469, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.0443, 0.02974, 0.01667, 0.01667, 0.05967, 0.04331, 0.06667, 0.01667, 0.01667, 0.02625, 0.01667]
msp_epo_45_weights = [0.04444, 0.04379, 0.04444, 0.04444, 0.04444, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.0312, 0.02923, 0.04444, 0.03726, 0.04444, 0.01557, 0.01111, 0.04296, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444]

msp_epo_15 = dict(zip(tickers_15, msp_epo_15_weights))
msp_epo_30 = dict(zip(tickers_30, msp_epo_30_weights))
msp_epo_45 = dict(zip(tickers_45, msp_epo_45_weights))

### MAX Returns + EPO weight

In [11]:
max_return_epo_15_weights = [0.13333, 0.03333, 0.13333, 0.03333, 0.13333, 0.03333, 0.03333, 0.03333, 0.03333, 0.03333, 0.03333, 0.13333, 0.13333, 0.03333, 0.03333]
max_return_epo_30_weights = [0.06667, 0.01667, 0.06667, 0.06667, 0.06667, 0.01667, 0.01667, 0.01667, 0.06667, 0.06667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667, 0.06667, 0.01667, 0.06667, 0.01667, 0.01667, 0.01667, 0.01667]
max_return_epo_45_weights = [0.04444, 0.04444, 0.04444, 0.04444, 0.04444, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.04444, 0.04444, 0.04444, 0.01111, 0.04444, 0.01111, 0.01111, 0.04444, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.01111, 0.04444]

max_return_epo_15 = dict(zip(tickers_15, max_return_epo_15_weights))
max_return_epo_30 = dict(zip(tickers_30, max_return_epo_30_weights))
max_return_epo_45 = dict(zip(tickers_45, max_return_epo_45_weights))

-----

### Rebalncing

In [12]:
# 분기별 (3개월) 리밸런싱
# LookBack period : 1year
# out-of-sample 기간 : 2023-05-01 ~ 2023-10-10

In [13]:
# in-sample price data
mkt = pd.read_csv('../new_market.csv')
mkt.set_index('pricingDate', inplace=True)
mkt.index = pd.to_datetime(mkt.index)
mkt_in = mkt.loc[:'2023-04-30', :]
mkt_out = mkt.loc['2023-05-01':, :]

In [14]:
mkt_in

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2018-05-01,66.24,43.36,42.2750,102.07,58.82,26.876666,151.92,224.08,88.86,45.03,...,191.85,46.65,76.95,50.64,70.37,86.64,116.95,136.78,55.05,83.40
2018-05-02,65.91,42.24,44.1425,100.37,57.85,26.310000,150.71,221.10,88.30,44.43,...,191.03,46.54,76.80,50.59,68.88,80.20,117.46,133.12,55.19,79.90
2018-05-03,66.34,42.11,44.2225,100.28,57.93,25.983333,151.73,226.05,87.91,43.60,...,191.62,46.48,76.54,49.24,69.63,82.67,115.18,133.54,54.45,80.38
2018-05-04,67.00,42.86,45.9575,100.17,58.74,26.276666,153.44,228.51,90.27,43.86,...,192.41,46.39,76.90,49.99,70.08,82.43,116.22,134.67,55.46,81.56
2018-05-07,67.39,42.44,46.2900,99.63,59.32,26.376666,152.30,230.99,90.50,43.69,...,191.91,46.13,77.74,46.92,70.02,83.11,116.99,137.48,55.51,82.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-24,138.48,13.32,165.3300,164.08,110.40,72.510000,277.25,377.34,186.96,80.80,...,114.48,71.03,118.20,41.44,104.54,138.90,138.14,288.19,28.27,176.35
2023-04-25,130.37,12.84,163.7700,164.90,109.97,72.590000,270.70,369.59,181.14,75.90,...,111.63,71.18,116.52,40.97,102.92,137.81,139.06,284.10,26.73,173.02
2023-04-26,133.12,12.74,163.7600,161.80,108.75,72.290000,271.21,363.06,180.66,77.03,...,109.45,69.68,115.45,40.93,100.98,138.01,139.08,280.42,26.39,172.94


In [15]:
mkt_out

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2023-05-01,136.08,13.89,169.59,153.71,111.11,75.90,279.42,374.15,182.54,77.80,...,115.60,70.17,114.67,42.39,104.50,142.23,138.83,287.45,26.82,179.56
2023-05-02,134.52,13.77,168.54,151.62,111.50,76.03,276.74,368.66,183.66,75.52,...,113.20,69.10,110.10,42.10,104.52,142.90,143.01,255.44,23.92,176.90
2023-05-03,134.77,13.79,167.45,149.25,111.76,75.62,275.18,345.25,182.39,75.02,...,109.33,68.50,107.93,40.48,104.15,137.31,139.62,258.50,22.66,177.73
2023-05-04,133.88,13.51,165.79,147.36,110.86,72.92,266.50,335.83,181.33,74.22,...,109.25,69.11,106.04,40.85,106.33,136.32,136.63,265.00,19.93,178.47
2023-05-05,133.30,13.87,173.57,148.03,111.26,74.81,265.65,348.40,183.39,75.96,...,110.78,69.57,108.68,41.30,108.97,137.49,137.91,273.80,23.76,186.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-04,111.67,12.73,173.66,147.69,95.65,80.46,309.39,518.42,173.79,73.47,...,88.21,56.63,111.50,33.30,91.22,123.53,109.69,230.94,33.33,170.99
2023-10-05,110.35,12.85,174.91,147.45,96.20,81.64,309.66,516.44,172.10,73.13,...,89.49,56.48,108.99,32.18,90.14,121.84,109.30,222.54,33.40,172.00
2023-10-06,110.64,12.76,177.49,148.24,96.88,82.18,312.19,526.68,173.97,72.86,...,92.41,57.35,107.17,32.48,90.97,119.46,110.91,223.85,33.69,175.58


### out-of-sample rebalancing

In [16]:
# # GPT가 짜준거(계속 오류남;;)
# # 수익률 및 공분산 계산 함수
# def calculate_return_covariance(data, lookback_period):
#     return data.pct_change(lookback_period).mean(), data.pct_change(lookback_period).cov()

# # EPO 최적화 함수
# def gmv_epo(dict):
#     data = yf.download(list(dict.keys()), start="2023-05-01", end="2023-10-11", interval='1wk')['Adj Close']
#     ret = data.pct_change().dropna()
    
#     num_assets = data.shape[1] # 종목 개수가 나와야함
#     weights = cp.Variable((num_assets,1)) # 종목 개수랑 같아야함
#     # corr_mat = np.corr(ret.values.T) # 개별 종목 별 기대수익률을 구해서 Correlation Matrix를 만듬
#     corr_mat = ret.corr()

#     theta = 0.75 # shrinkage parameter
#     identity_matrix = np.eye(corr_mat.shape[0])  # identity matrix
#     corr_mat = (1-theta) * corr_mat + theta * identity_matrix # corr_mat에 shrinkage parameter 적용

#     print(corr_mat)
    
#     obj = cp.Minimize(cp.quad_form(weights, corr_mat)) # 목적식 설정
    
#     # sum(weights) = 1, weights >= 0, min weight ~ max weight 범위 제약
#     const = [cp.sum(weights) == 1, weights >= 0, ((1/num_assets))/2 <= weights, weights <= 2*(1/num_assets)] # 제약식 설정, 공매도 원하면 위에 주석 가져다 옆으로 복붙하면됨
    
#     problem = cp.Problem(obj, const) # 문제 정의
#     problem.solve(verbose=False, solver=cp.CPLEX) # 문제 풀기
    
#     if problem.status == "optimal":
#         w_opt = np.array(weights.value).flatten()
#         print("Optimal")
#     else:
#         print("It isn't optimal")

#     return w_opt

# # 포트폴리오 리턴 계산 함수
# def calculate_portfolio_return(weights, data):
#     weighted_returns = (data.pct_change() * weights).sum(axis=1)
#     return weighted_returns

# # def calculate_portfolio_return(weights, data):
# #     # 데이터의 컬럼 수와 가중치 벡터의 길이가 동일한지 확인
# #     if len(weights) != data.shape[1]:
# #         raise ValueError("Length of weights does not match number of columns in data")

# #     weighted_returns = (data.pct_change() * weights).sum(axis=1)
# #     return weighted_returns

# # 리밸런싱 및 성과 평가 프로세스
# def rebalancing_process(data, rebalancing_period, lookback_period):
#     performance = []
#     rebalancing_dates = data.resample(rebalancing_period).mean().index

#     for date in rebalancing_dates:
#         # Lookback 기간에 대한 데이터 선택
#         lookback_data = data.loc[date - timedelta(days=lookback_period):date]

#         # 수익률과 공분산 계산
#         returns, covariance = calculate_return_covariance(lookback_data, lookback_period)

#         # EPO 최적화
#         optimal_weights = gmv_epo(stocks_15)

#         # 리밸런싱 기간 동안의 포트폴리오 리턴 계산
#         portfolio_return = calculate_portfolio_return(optimal_weights, data.loc[date:date + timedelta(days=90)])
#         performance.append(portfolio_return)

#     # 전체 성과 평가
#     total_performance = pd.concat(performance).cumsum()
#     return total_performance

# # 가상 데이터 생성 및 리밸런싱 프로세스 실행
# portfolio_performance = rebalancing_process(mkt_out, '3M', 365)

# # 결과 출력 (일부만 출력)
# portfolio_performance.head()

In [17]:
# 예시 데이터
# # 자산별 현재 가격과 포트폴리오 내 비중
# current_prices = {"Asset1": 100, "Asset2": 200, "Asset3": 300}
# current_weights = {"Asset1": 0.4, "Asset2": 0.4, "Asset3": 0.2}

# # 자산별 과거 가격 데이터 (예시)
# price_data = {
#     "Asset1": [95, 100, 105],
#     "Asset2": [195, 200, 205],
#     "Asset3": [295, 300, 305]
# }
# price_df = pd.DataFrame(price_data)

# # 수익률과 공분산 행렬 계산
# mu = expected_returns.mean_historical_return(price_df)
# S = risk_models.sample_cov(price_df)

# # 효율적인 프론티어 계산
# ef = EfficientFrontier(mu, S)
# weights = ef.max_sharpe()

# # 새로운 자산 배분
# new_weights = {k: weights[k] for k in current_weights.keys()}

# # 리밸런싱 필요 금액 계산
# portfolio_value = sum(current_prices[k] * current_weights[k] for k in current_prices)
# new_allocation = {k: portfolio_value * new_weights[k] for k in new_weights}
# rebalancing_trades = {k: new_allocation[k] - current_prices[k] * current_weights[k] for k in current_prices}

# print("새로운 자산 배분:", new_weights)
# print("리밸런싱을 위한 거래:", rebalancing_trades)

In [42]:
a = mkt_out.loc['2023-07-31':'2023-07-31', tickers_15]
b = [] # 종가 저장 리스트
for i in a.columns:
    b.append(float(a[i]))

b

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_11420\3624546341.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  b.append(float(a[i]))


[196.45,
 133.68,
 467.29,
 157.96,
 156.3,
 36.06,
 167.53,
 61.93,
 107.24,
 73.3,
 132.72,
 335.92,
 267.43,
 110.39,
 32.0]

In [73]:
## market 데이터 불러오기
def downloads(dict, start_date, end_date):
    data = pd.DataFrame()
    a = pd.DataFrame()
    tickers = list(dict.keys())

    for ticker in tqdm(tickers):
        a = mkt.loc[start_date:end_date, ticker]
        data = pd.concat([data, a], axis=1)

    return data

In [102]:
# 초기 투자 금액 (1만 달러)
initial = 10000

# 초기 가중치 w* : GMW+EPO weight
## 종목 리스트
tickers_15
tickers_30
tickers_45
## 가중치 리스트
gmv_epo_15_weights
gmv_epo_30_weights
gmv_epo_45_weights
## 종목 : 가중치 딕셔너리
gmv_epo_15
gmv_epo_30
gmv_epo_45

# w* 이용하여 2023-05-01 부터 2023-07-31까지(3개월간) 투자 후 payoff(2023-07-31의 종가 * w*) 확인 
a = mkt_out.loc['2023-07-31':'2023-07-31', tickers_15] # 2023-07-31 종가 df
b = [] # 2023-07-31 종가 저장 리스트
for i in a.columns:
    b.append(float(a[i]))
# payoff_med_15 = sum(np.multiply(b, gmv_epo_15_weights)) 

# w* 이용하여 2023-05-01 당시의 초기 투자량(payoff) 확인
c = mkt_out.loc['2023-05-01':'2023-05-01', tickers_15] # 2023-05-01 종가 df
d = [] # 2023-05-01 종가 저장 리스트
for i in c.columns:
    d.append(float(c[i]))
k = list((np.divide(np.multiply(gmv_epo_15_weights, [initial] * 15), d))) # 초기 투자 주식 보유량(XX주)

# 3개월 동안의 투자 수익률 확인 (소수점 단위)
ret_3m = ((sum(np.multiply(k,b)) - initial)/initial)   # ((2023-07-31의 payoff - 초기 payoff) / 초기 payoff)

# 3개월 후 나의 총 투자 (손)수익
total_ret_3m = initial*(1+ret_3m)  # 달러

# 2022-08-01~2023-07-31까지(lookback = 1yr)의 데이터로 EPO 최적화
## 리밸런싱을 위한 GMV+EPO 최적화
def gmv_epo(dict):

    data = downloads(dict, "2022-08-01", "2023-07-31")  # mkt 데이터에서 일별 주가데이터 가져오기
    ret = data.pct_change().dropna()
    
    num_assets = data.shape[1] # 종목 개수가 나와야함
    weights = cp.Variable((num_assets,1)) # 종목 개수랑 같아야함
    # corr_mat = np.corr(ret.values.T) # 개별 종목 별 기대수익률을 구해서 Correlation Matrix를 만듬
    corr_mat = ret.corr()

    theta = 0.75 # shrinkage parameter
    identity_matrix = np.eye(corr_mat.shape[0])  # identity matrix
    corr_mat = (1-theta) * corr_mat + theta * identity_matrix # corr_mat에 shrinkage parameter 적용

    print(corr_mat)
    
    obj = cp.Minimize(cp.quad_form(weights, corr_mat)) # 목적식 설정
    
    # 가중치 합 = 1, weights >= 0, min weight ~ max weight 범위 제약
    const = [cp.sum(weights) == 1, weights >= 0, ((1/num_assets))/2 <= weights, weights <= 2*(1/num_assets)] # 제약식 설정, 공매도 원하면 위에 주석 가져다 옆으로 복붙하면됨
    
    problem = cp.Problem(obj, const) # 문제 정의
    problem.solve(verbose=False, solver=cp.CPLEX) # 문제 풀기
    
    if problem.status == "optimal":
        w_opt = np.array(weights.value).flatten()
        print("Optimal")
        # print(gmv_weights.value[:5])
        
        # print(max(gmv_weights.value)) # 가장 큰 weight를 보고싶으면 이거 주석 해제
    else:
        print("It isn't optimal")

    return w_opt
gmv_epo_15_rebal = list(gmv_epo(stocks_15))  # 리밸런싱 시점에서의 최적 가중치 w**

# w** 이용하여 2023-08-01 부터 2023-10-10까지(3개월간) 투자 후 payoff(2023-10-10의 종가 * w**) 확인 
aa = mkt_out.loc['2023-10-10':'2023-10-10', tickers_15] # 2023-10-10 종가 df
bb = [] # 2023-10-10 종가 저장 리스트
for i in aa.columns:
    bb.append(float(aa[i]))
# payoff_final_15 = sum(np.multiply(bb, gmv_epo_15_rebal)) 

# w** 이용하여 2023-08-01 당시의 투자량(payoff) 확인
cc = mkt_out.loc['2023-08-01':'2023-08-01', tickers_15] # 2023-08-01 종가 df
dd = [] # 2023-08-01 종가 저장 리스트
for i in cc.columns:
    dd.append(float(cc[i]))
kk = list((np.divide(np.multiply(gmv_epo_15_rebal, [total_ret_3m] * 15), dd))) # 2023-08-01 기준 주식 보유량(XX주)

# 3개월 동안의 투자 수익률 확인 (소수점 단위)
ret_rebal_3m = ((sum(np.multiply(kk,bb)) - initial)/initial)   # ((2023-07-31의 payoff - 초기 payoff) / 초기 payoff)

# 2023-10-10 기준 나의 총 투자 (손)수익
total_ret_rebal_3m = initial*(1+ret_rebal_3m)  # 달러
total_ret_rebal_3m

C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_11420\46256481.py:22: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  b.append(float(a[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_11420\46256481.py:29: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  d.append(float(c[i]))
100%|██████████| 15/15 [00:00<00:00, 681.68it/s]

           AAPL      AMZN      NVDA       JPM        PG       PFE       JNJ  \
AAPL   1.000000  0.156857  0.148238  0.109607  0.123385  0.080690  0.074690   
AMZN   0.156857  1.000000  0.130122  0.087259  0.086409  0.053851  0.040523   
NVDA   0.148238  0.130122  1.000000  0.096845  0.063929  0.035674  0.021050   
JPM    0.109607  0.087259  0.096845  1.000000  0.091847  0.084323  0.066647   
PG     0.123385  0.086409  0.063929  0.091847  1.000000  0.105001  0.114342   
PFE    0.080690  0.053851  0.035674  0.084323  0.105001  1.000000  0.122511   
JNJ    0.074690  0.040523  0.021050  0.066647  0.114342  0.122511  1.000000   
KO     0.129419  0.088359  0.080624  0.102554  0.179366  0.104170  0.118525   
XOM    0.081365  0.036812  0.044969  0.100556  0.033779  0.046001  0.031164   
NEE    0.124011  0.083511  0.071494  0.083373  0.131879  0.089971  0.083161   
GOOGL  0.171887  0.174978  0.130407  0.078736  0.087159  0.065198  0.056174   
MSFT   0.179314  0.175678  0.164037  0.084968  0.109

Optimal


C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_11420\46256481.py:80: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  bb.append(float(aa[i]))
C:\Users\Yoojiyoung\AppData\Local\Temp\ipykernel_11420\46256481.py:87: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  dd.append(float(cc[i]))


10625.719208786542

In [103]:
ret_3m, total_ret_3m

(0.13032659312335526, 11303.265931233553)

In [104]:
ret_rebal_3m, total_ret_rebal_3m

(0.06257192087865424, 10625.719208786542)

In [105]:
[total_ret_3m] * 15

[11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553,
 11303.265931233553]

In [97]:
np.multiply(gmv_epo_15_rebal, [total_ret_3m] * 15)

array([ 376.81584007,  676.04287379,  784.56292215,  665.37866737,
        716.41585322,  927.99271088, 1054.98883169,  606.71861313,
       1148.2194066 ,  761.65738615,  666.03937158,  564.18275499,
       1035.07874463,  638.97049112,  680.20146385])

In [98]:
np.divide(np.multiply(gmv_epo_15_rebal, [total_ret_3m] * 15), dd)

array([ 1.92641211,  5.13359309,  1.68697814,  4.2332273 ,  4.58506146,
       26.05989079,  6.2466033 ,  9.82222136, 10.76926849, 10.55365645,
        5.06301309,  1.67741796,  3.9647556 ,  5.84068091, 21.51174775])